<a href="https://colab.research.google.com/github/jamilly-b/curso-machine-learning/blob/main/Projeto/letterboxd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np

In [2]:
drive.mount('/content/drive')
dados_filmes = pd.read_csv('/content/drive/MyDrive/machine-learning-project/movie_data.csv', lineterminator='\n')
usuario = pd.read_csv('/content/drive/MyDrive/machine-learning-project/users_export.csv')
avaliacao = pd.read_csv('/content/drive/MyDrive/machine-learning-project/ratings_export.csv')

Mounted at /content/drive


## Visualização dos dados

### Significado das variáveis

**_id** = ID exclusivo global.

**display_name** = Nome de exibição em Letterboxd no momento do último rastreamento. /// **num_ratings_pages** = Número de páginas de avaliações ///  **num_reviews** = Número total de avaliações feitas por este usuário. /// **username** = Letterboxd ID ///nome de usuário exclusivo. Pode ser usado para formar um URL Letterboxd. /// **_id** = ID exclusivo global. /// **genres** = Array  de gêneros de filmes associados. /// **image_url** = Imagem usada no site Letterboxd para o filme. Geralmente um pôster. /// **imdb_id** = ID para IMDB. /// **imdb_link** = Link para a página IMDB do filme. /// **movie_id** = Identificação do filme Letterboxd. /// **movie_title** = Título de exibição. ///**original_language** = Idioma original do filme /// **overview** = Descrição do filme /// **popularity** = Popularidade numérica  /// **production_countries** = Matriz de países em que o filme foi produzido /// **release_date** = Data de lançamento /// **runtime** = Tempo de execução em minutos /// **spoken_languages** = Array de todos os idiomas falados no filme /// **tmdb_id** = ID para acessar dados na API /// **tmdb_link** = Link para a entrada do filme no TMDB ///**vote_average** = Média de votos /// **vote_count** = contagem de votos ///**year_released** = Ano lançado /// **_id** = ID exclusivo global. /// **movie_id** = Letterboxd ID exclusivo para o filme. Pode ser usado para corresponder aos dados do filme neste conjunto de dados ou para construir um Letterboxd /// **rating_val** = Valor da classificação com base nas estrelas dadas (em incrementos de meia estrela). Uma meia estrela é 1, cinco estrelas é 10, etc. /// **user_id** = Letterboxd ID exclusivo para o usuário. Pode ser usado para corresponder aos dados do usuário neste conjunto de dados ou para construir um Letterboxd

### Tabela Usuário

In [3]:
usuario.head() 

,_id,display_name,num_ratings_pages,num_reviews,username
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof
1,5fc4172ec6cd28ebd99dd0ea,Matt Singer,52.0,1915,superpulse
2,5fc4172ec6cd28ebd99dd0ed,Sean Baker,21.0,1283,lilfilm
3,5fc4172ec6cd28ebd99dd0ee,iana,37.0,1177,ianamurray
4,5fc419171ebf67b9fbe48615,Lizzy,57.0,1810,punchdrunklizzy


In [4]:
usuario.isnull().sum()

_id                     0
display_name          307
num_ratings_pages    3392
num_reviews             0
username                0
dtype: int64

Limpando os dados nulos e selecionando os dados que serão úteis:

In [5]:
usuario = usuario.dropna(subset= ['display_name', 'num_ratings_pages']) #apagando os dados nulos

In [6]:
usuario.shape

(4747, 5)

In [7]:
usuario = usuario.drop(usuario[usuario.num_reviews < 1000].index) #selecionando os usuários que possuem mais de 1000 avaliações

In [8]:
usuario = usuario.drop(columns = ['num_reviews', 'num_ratings_pages', 'display_name'])

In [9]:
usuario.shape

(1867, 2)

Agora a quantidade de usuários que serão avaliados foi reduzido, para deixar o data frame mais leve.

### Tabela Dados dos filmes

In [10]:
dados_filmes.head(3)

,_id,genres,image_url,imdb_id,imdb_link,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc85f606758f69634496fd3,"[""Music"",""Animation""]",film-poster/4/6/4/4/4/0/464440-football-freaks...,NaN,NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],535272.0,https://www.themoviedb.org/movie/535272/,0.0,0.0,1971.0
1,5fc85ff26758f696344ace0c,[],film-poster/2/4/5/5/0/0/245500-aftermath-0-230...,tt0586129,http://www.imdb.com/title/tt0586129/maindetails,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,https://www.themoviedb.org/movie/318331/,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",film-poster/9/3/3/1/8/93318-where-chimneys-are...,tt0045731,http://www.imdb.com/title/tt0045731/maindetails,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,https://www.themoviedb.org/movie/117779/,6.6,10.0,1953.0


In [11]:
dados_filmes.columns

Index(['_id', 'genres', 'image_url', 'imdb_id', 'imdb_link', 'movie_id',
       'movie_title', 'original_language', 'overview', 'popularity',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'tmdb_id', 'tmdb_link', 'vote_average', 'vote_count', 'year_released'],
      dtype='object')

In [12]:
dados_filmes.shape

(285504, 19)

A tabela possui 285.504, é muito interessante reduzir essa quantidade, pois a maioria dos filmes não são relevantes para nosso projeto.

In [13]:
dados_filmes.isna().sum()

_id                         0
genres                  10632
image_url               21097
imdb_id                 41702
imdb_link               41702
movie_id                    2
movie_title              2164
original_language       10632
overview                24256
popularity              10632
production_countries    10632
release_date            14454
runtime                 15449
spoken_languages        10632
tmdb_id                  5587
tmdb_link                5587
vote_average            10632
vote_count              10632
year_released            7572
dtype: int64

Filtrando os dados interessantes:

In [14]:
dados_filmes = dados_filmes.drop(dados_filmes[dados_filmes.popularity < 9].index)
dados_filmes = dados_filmes.drop(dados_filmes[dados_filmes.vote_count < 1000].index)
dados_filmes = dados_filmes.drop(dados_filmes[dados_filmes.runtime < 50].index)
dados_filmes = dados_filmes.drop(dados_filmes[dados_filmes.year_released < 1940].index)

Tratando os dados nulos:

In [15]:
dados_filmes = dados_filmes.dropna(subset= ['genres', 'image_url', 'imdb_id', 'imdb_link', 'movie_id','movie_title', 'original_language', 'overview', \
                                            'popularity','production_countries', 'release_date', 'runtime', 'spoken_languages','tmdb_id', 'tmdb_link', 'vote_average', 'vote_count', 'year_released'])

Eliminando as colunas desnecessárias:

In [16]:
dados_filmes = dados_filmes.drop(columns= ['imdb_id', 'original_language', 'overview', 'release_date', 'runtime', 'spoken_languages','tmdb_id', 'tmdb_link'])

In [17]:
dados_filmes.shape

(3159, 11)

Conseguimos uma redução de 98% nessa tabela.

### Tabela Avaliação

Essa tabela é a maior de todas, e precisa ser o mais enxuta possível, senão o Colab não conseguirá rodar o treino. Contudo, não é possível fazer muita coisa agora, o tratamento dela será feito após o Merge.

In [18]:
avaliacao.head()

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof


In [19]:
avaliacao.shape

(11078167, 4)

In [20]:
avaliacao.columns

Index(['_id', 'movie_id', 'rating_val', 'user_id'], dtype='object')

In [21]:
avaliacao.isnull().sum()

_id           0
movie_id      6
rating_val    0
user_id       0
dtype: int64

In [22]:
avaliacao = avaliacao.dropna(subset = ['movie_id'])

## Manipulação dos dados

In [23]:
avaliacao = avaliacao.rename(columns={'user_id': 'username'})

In [24]:
tabela = pd.merge(usuario, avaliacao, on = 'username', how = 'inner')
tabela = pd.merge(tabela, dados_filmes, on = 'movie_id', how = 'inner')
tabela.head(3)

,_id_x,username,_id_y,movie_id,rating_val,_id,genres,image_url,imdb_link,movie_title,popularity,production_countries,vote_average,vote_count,year_released
0,5fc4172ec6cd28ebd99dd0e2,deathproof,5fc57c5c6758f69634519398,mank,5,5fc884286758f69634f3ceca,"[""Drama"",""History""]",film-poster/5/4/1/4/2/5/541425-mank-0-230-0-34...,http://www.imdb.com/title/tt10618286/maindetails,Mank,16.331,"[""United States of America""]",6.9,1077.0,2020.0
1,5fc4172ec6cd28ebd99dd0ea,superpulse,5fc57cb26758f69634541441,mank,6,5fc884286758f69634f3ceca,"[""Drama"",""History""]",film-poster/5/4/1/4/2/5/541425-mank-0-230-0-34...,http://www.imdb.com/title/tt10618286/maindetails,Mank,16.331,"[""United States of America""]",6.9,1077.0,2020.0
2,5fc4172ec6cd28ebd99dd0ee,ianamurray,5fd05004b89a841f5b220ee5,mank,7,5fc884286758f69634f3ceca,"[""Drama"",""History""]",film-poster/5/4/1/4/2/5/541425-mank-0-230-0-34...,http://www.imdb.com/title/tt10618286/maindetails,Mank,16.331,"[""United States of America""]",6.9,1077.0,2020.0


Quando as tabelas foram unidas com o Merge, o ID de cada uma ficou confuso, então é necessário renomear cada um:

In [25]:
tabela = tabela.rename(columns={'_id_x': 'id_usuario','_id': 'id_filme','_id_y': 'id_avaliacao'})

In [26]:
tabela.columns

Index(['id_usuario', 'username', 'id_avaliacao', 'movie_id', 'rating_val',
       'id_filme', 'genres', 'image_url', 'imdb_link', 'movie_title',
       'popularity', 'production_countries', 'vote_average', 'vote_count',
       'year_released'],
      dtype='object')

In [27]:
tabela.shape

(1910127, 15)

In [28]:
tabela.isna().sum()

id_usuario              0
username                0
id_avaliacao            0
movie_id                0
rating_val              0
id_filme                0
genres                  0
image_url               0
imdb_link               0
movie_title             0
popularity              0
production_countries    0
vote_average            0
vote_count              0
year_released           0
dtype: int64

Escolhendo cada tributo da tabela que iremos utilizar no treinamento e transformando em um vetor para melhor manipulação dos dados

In [29]:
df = pd.DataFrame() # Criando um data frame vazio

As variáveis que consideramos interessantes para avaliar são: 'id_usuario', 'username', 'rating_val', 'id_filme', 'genres', 'movie_title', 'popularity', 'production_countries', 'vote_average', 'year_released'.

In [30]:
vetor_id_usuario = np.array(tabela.id_usuario)
vetor_username = np.array(tabela.username)
vetor_rating_val = np.array(tabela.rating_val)
vetor_id_filme = np.array(tabela.id_filme)
vetor_genres = np.array(tabela.genres)
vetor_movie_title = np.array(tabela.movie_title)
vetor_popularity = np.array(tabela.popularity)
vetor_production_countries = np.array(tabela.production_countries)
vetor_vote_average = np.array(tabela.vote_average)
vetor_year_released = np.array(tabela.year_released)

In [31]:
Colunas =['ID_usuario','Username','Nota_usuario','ID_Filme','Generos','Nome_Filme','Popularidade','País_Produzido', 'Nota_Media', 'Ano_lançamento']

In [32]:
df = pd.DataFrame(columns = Colunas)

df.ID_usuario = vetor_id_usuario
df.Username = vetor_username
df.Nota_usuario = vetor_rating_val
df.ID_Filme = vetor_id_filme
df.Generos = vetor_genres
df.Nome_Filme = vetor_movie_title
df.Popularidade = vetor_popularity
df.País_Produzido = vetor_production_countries
df.Nota_Media = vetor_vote_average
df.Ano_lançamento = vetor_year_released

Criando uma função para avaliar se o usuário gostou ou não do filme de acordo com a nota de avaliação.

In [59]:
def classificar_nota(nota):
  if nota >= 7:
    return 1  # Se a nota de avalição do usuário for maior ou igual a 7, consideramos que ele gostou desse filme
  else:
    return 0  # Se a nota de avalição do usuário for menor 7, consideramos que ele não gostou desse filme
df['Recomendacao'] = df['Nota_usuario'].apply(classificar_nota)
print(df)

                 ID_usuario  Username  Nota_usuario            ID_Filme  \
0              541726289902       423             5    5884286758696343   
1               54172628990      1602             6    5884286758696343   
2               54172628990       752             7    5884286758696343   
3           541917167948615      1350             5    5884286758696343   
4              541726289908       343             5    5884286758696343   
...                     ...       ...           ...                 ...   
1910122  598651418738210547       799             5     600798159536794   
1910123     598614187382109      1549             7  586675869634908337   
1910124     598141873821116        51             8  586675869634908337   
1910125   60061315953679982      1075             3  588586758696349370   
1910126  608811528290115283       711             3    6088892829011572   

         Generos         Nome_Filme  Popularidade  País_Produzido  Nota_Media  \
0            574  

In [61]:
df['Recomendacao'].value_counts()

1    1056906
0     853221
Name: Recomendacao, dtype: int64

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1910127 entries, 0 to 1910126
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   ID_usuario      object 
 1   Username        object 
 2   Nota_usuario    int64  
 3   ID_Filme        object 
 4   Generos         object 
 5   Nome_Filme      object 
 6   Popularidade    float64
 7   País_Produzido  object 
 8   Nota_Media      float64
 9   Ano_lançamento  float64
 10  Recomendacao    int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 160.3+ MB


ID_usuario, Username, ID_Filme, Genero, Nome_filme, País_Produzido são objetos, e precisam ser convertidos para números.

In [35]:
from sklearn.preprocessing import LabelEncoder

Removendo os elemento não numéricos:

In [36]:
df['ID_Filme'] = df['ID_Filme'].str.replace(r'\D', '')
df['ID_usuario'] = df['ID_usuario'].str.replace(r'\D', '')

<ipython-input-36-35bc5663e369>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ID_Filme'] = df['ID_Filme'].str.replace(r'\D', '')
<ipython-input-36-35bc5663e369>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['ID_usuario'] = df['ID_usuario'].str.replace(r'\D', '')


In [37]:
le = LabelEncoder()

In [38]:
df['Generos'] = le.fit_transform(df['Generos'])
df['Username'] = le.fit_transform(df['Username'])
df['País_Produzido'] = le.fit_transform(df['País_Produzido'])

## Árvore de decisão

In [39]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [62]:
amostra = df.sample(frac = 0.60)

In [63]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

In [76]:
X = amostra[['ID_usuario','Username','ID_Filme','Generos','Popularidade','País_Produzido', 'Nota_Media', 'Ano_lançamento']]
y = amostra['Recomendacao']

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [83]:
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [72]:
from sklearn.metrics import accuracy_score

In [88]:
y_pred = model.predict(X_test)

In [89]:
accuracy = accuracy_score(y_test, y_pred)

print(f'Precisão: {accuracy:.2f}')

Precisão: 0.66


In [90]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.62      0.62      0.62    102267
           1       0.69      0.69      0.69    126949

    accuracy                           0.66    229216
   macro avg       0.66      0.66      0.66    229216
weighted avg       0.66      0.66      0.66    229216



## Random Forest

In [91]:
from sklearn.ensemble import RandomForestClassifier

In [98]:
from sklearn.utils.extmath import randomized_range_finder
RandomForestClassifier(random_state = 42)

RandomForestClassifier(random_state=42)

In [99]:
model = RandomForestClassifier(random_state = 42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [105]:
predictions_train = model.predict(X_train)
accuracy = metrics.accuracy_score(y_train, predictions_train)
print('Accuracy:%.4f' %accuracy)

Accuracy:0.9999


In [104]:
predictions_test = model.predict(X_test)
accuracy = metrics.accuracy_score(y_test, predictions_test)
print('Accuracy:%.4f' %accuracy)

Accuracy:0.7035


In [106]:
print(metrics.classification_report(y_test, predictions_test))

              precision    recall  f1-score   support

           0       0.68      0.65      0.66    102267
           1       0.72      0.75      0.74    126949

    accuracy                           0.70    229216
   macro avg       0.70      0.70      0.70    229216
weighted avg       0.70      0.70      0.70    229216

